# Shelf animations

In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext watermark

import cf_xarray
import dask
import dcpy
import distributed
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.units as munits
import ncar_jobqueue
import numpy as np
import pandas as pd
import xarray as xr
import xgcm

mpl.rcParams["savefig.dpi"] = 300
mpl.rcParams["savefig.bbox"] = "tight"
mpl.rcParams["figure.dpi"] = 180

munits.registry[np.datetime64] = mdates.ConciseDateConverter()

xr.set_options(keep_attrs=True, display_style="html")


%watermark -iv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dask         : 2.30.0
dcpy         : 0.1
xgcm         : 0.5.1.dev91+g86c9641
pandas       : 1.1.3
xarray       : 0.16.3.dev150+g37522e991
numpy        : 1.19.2
cf_xarray    : 0.4.1.dev31+g7a8c620
matplotlib   : 3.3.2
distributed  : 2.30.0
ncar_jobqueue: 2020.3.4



In [27]:
def inline_getitem(obj: xr.DataArray):

    import operator

    import dask

    obj = dask.optimize(obj)[0]
    obj.data.dask = dask.optimization.inline_functions(
        obj.data.dask, [], fast_functions=[operator.getitem]
    )
    return obj


def div_stencil(U, V, grid):  # Basic stencil operator for divergence

    if grid.arakawa == "A":
        U = grid.interp(U, axis="X", boundary="extend")
        V = grid.interp(V, axis="Y", boundary="extend")

    elif grid.arakawa == "B":
        U = grid.interp(U, axis="Y", boundary="extend")
        V = grid.interp(V, axis="X", boundary="extend")

    dUdx = grid.diff(U, axis="X", boundary="extend")
    dVdy = grid.diff(V, axis="Y", boundary="extend")

    return dUdx + dVdy


def divh(U, V, grid, boundary=None):

    UT = U * grid.get_metric(U, "YZ")
    VT = V * grid.get_metric(V, "XZ")

    div = div_stencil(UT, VT, grid)
    area = grid.get_metric(div, "XY")

    return div / area


def curlz(U, V, grid):

    Udx = U * grid.get_metric(U, "X")
    Vdy = V * grid.get_metric(V, "Y")

    der = div_stencil(Vdy, -Udx, grid)
    area = grid.get_metric(der, "XY")

    return der / area


def map_xgcm_func(ds, func, *args, **kwargs):
    dsnew = xr.Dataset()
    for var in ds:
        if "xq" in ds[var].dims and "yq" in ds[var].dims:
            dsnew[var] = ds[var]
            continue
        try:
            dsnew[var] = func(ds[var], *args, **kwargs)
        except ValueError as e:
            dsnew[var] = ds[var]
            pass
    return dsnew


def xgcm_grid_mom6(geometry):
    import xgcm
    
    grid = xgcm.Grid(
        geometry,
        coords={
            "X": {"center": "xh", "outer": "xq"},
            "Y": {"center": "yh", "outer": "yq"},
            "Z": {"inner": "z_l", "outer": "z_i"},
        },
        periodic=False,
        metrics={
            ("X",): ["dxT", "dxCu", "dxCv", "dxBu"],  # X distances
            ("Y",): ["dyT", "dyCu", "dyCv", "dyBu"],  # Y distances
            # ("Z",): ["drW", "drS", "drC"],  # Z distances
            ("X", "Y"): ["Ah", "Aq"],  # Areas
        },
        boundary={"X": "extend", "Y": "extend", "Z": "extend"}
    )
    grid.arakawa = "C"
    return grid

In [3]:
if "client" in locals():
    client.close()
if "cluster" in locals():
    cluster.close()

env = {"OMP_NUM_THREADS": "3", "NUMBA_NUM_THREADS": "3"}

# cluster = distributed.LocalCluster(
#    n_workers=8,
#    threads_per_worker=1,
#    env=env
# )

if "cluster" in locals():
    del cluster

cluster = ncar_jobqueue.NCARCluster(
    project="NCGD0048",
    scheduler_options=dict(dashboard_address=":9797"),
)
# cluster = dask_jobqueue.PBSCluster(
#    cores=9, processes=9, memory="108GB", walltime="02:00:00", project="NCGD0043",
#    env_extra=env,
# )

cluster.scale(25)

client = distributed.Client(cluster)
client

Client Scheduler: tcp://10.12.205.14:32793 Dashboard: https://jupyterhub.ucar.edu/dav/user/dcherian/proxy/9797/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
dirname = "/glade/p/nsc/ncgd0048/ETP_1_20/"
path = f"{dirname}/COMPRESSED_OCEAN_SHELF/ocean_shelf_20*.nc"

In [179]:
rawds = xr.open_mfdataset(
    path,
    parallel=True,
    chunks={"time": -1, "zl": -1, "zi": -1},
)

In [180]:
rawds

<xarray.Dataset>
Dimensions:           (time: 55520, xh: 301, xq: 301, yh: 241, yq: 241, zi: 141, zl: 140)
Coordinates:
  * xq                (xq) float64 268.0 268.1 268.1 268.1 ... 282.9 282.9 283.0
  * yh                (yh) float64 1.975 2.025 2.075 2.125 ... 13.88 13.93 13.98
  * zl                (zl) float64 1.25 3.75 6.25 ... 6.324e+03 6.475e+03
  * time              (time) datetime64[ns] 1999-12-30T01:30:00 ... 2018-12-2...
    average_T1        (time) datetime64[ns] dask.array<chunksize=(8,), meta=np.ndarray>
    average_T2        (time) datetime64[ns] dask.array<chunksize=(8,), meta=np.ndarray>
    average_DT        (time) timedelta64[ns] dask.array<chunksize=(8,), meta=np.ndarray>
  * xh                (xh) float64 268.0 268.0 268.1 268.1 ... 282.9 282.9 283.0
  * yq                (yq) float64 2.0 2.05 2.1 2.15 ... 13.85 13.9 13.95 14.0
  * zi                (zi) float64 0.0 2.5 5.0 ... 6.199e+03 6.449e+03 6.5e+03
Data variables: (12/20)
    uo                (time, zl, yh, xq) float32 dask.array<chunksize=(8, 140, 241, 301), meta=np.ndarray>
    vo                (time, zl, yq, xh) float32 dask.array<chunksize=(8, 140, 241, 301), meta=np.ndarray>
    zos               (time, yh, xh) float32 dask.array<chunksize=(8, 241, 301), meta=np.ndarray>
    thetao            (time, zl, yh, xh) float32 dask.array<chunksize=(8, 140, 241, 301), meta=np.ndarray>
    so                (time, zl, yh, xh) float32 dask.array<chunksize=(8, 140, 241, 301), meta=np.ndarray>
    taux              (time, yh, xq) float32 dask.array<chunksize=(8, 241, 301), meta=np.ndarray>
    ...                ...
    Kd_shear          (time, zi, yh, xh) float32 dask.array<chunksize=(8, 141, 241, 301), meta=np.ndarray>
    Kv_u              (time, zl, yh, xq) float32 dask.array<chunksize=(8, 140, 241, 301), meta=np.ndarray>
    ePBL_h_ML         (time, yh, xh) float32 dask.array<chunksize=(8, 241, 301), meta=np.ndarray>
    bbl_thick_u       (time, yh, xq) float32 dask.array<chunksize=(8, 241, 301), meta=np.ndarray>
    kv_bbl_u          (time, yh, xq) float32 dask.array<chunksize=(8, 241, 301), meta=np.ndarray>
    Kd_BBL            (time, zi, yh, xh) float32 dask.array<chunksize=(8, 141, 241, 301), meta=np.ndarray>

## Make plot

In [81]:
def inset_colorbar(
    axes, mappable, orientation="horizontal", inset_kwargs=None, cbar_kwargs=None
):

    from mpl_toolkits.axes_grid1.inset_locator import inset_axes

    if inset_kwargs is None:
        inset_kwargs = {}
    if cbar_kwargs is None:
        cbar_kwargs = {}

    cbar_kwargs["orientation"] = orientation

    if orientation == "horizontal":
        ikwargs = {"width": "50%", "height": "5%", "loc": "upper right"}
    else:
        ikwargs = {"width": "5%", "height": "50%", "loc": "upper right"}
    for k, v in ikwargs.items():
        inset_kwargs.setdefault(k, v)

    axins1 = inset_axes(axes, **ikwargs)
    axes.get_figure().colorbar(mappable, cax=axins1, **cbar_kwargs)
    return axins1


def plot_ts(da, ax, vmin, vmax, levels, cmap):

    
    ckwargs = dict(orientation="vertical", cbar_kwargs={"extend": "neither"})
    ht = da.plot(ax=ax, vmin=vmin, vmax=vmax, add_colorbar=False, cmap=cmap)
    da.plot.contour(ax=ax, levels=levels, colors="w")
    da.plot.contour(ax=ax, levels=levels, colors="k", linewidths=0.5)
    inset_colorbar(ax, ht, **ckwargs)


def plot_frame(sub):
    import cf_xarray
    import dcpy
    import matplotlib as mpl
    import matplotlib.pyplot as plt

    f0 = dcpy.oceans.coriolis(sub.vor.cf["Y"])
    mpl.use("agg")

    ckwargs = dict(orientation="vertical", cbar_kwargs={"extend": "neither"})

    returnval = sub.time
    sub = sub.squeeze()

    f, axx = plt.subplots(2, 3, sharex=True, sharey=True, constrained_layout=True)

    stride = 4
    wstride = int(np.ceil(stride * 1.5))

    ax = {
        "wind": axx[0, 0],
        "sst": axx[0, 1],
        "sss": axx[0, 2],
        "usurf": axx[1, 0],
        "ubot": axx[1, 1],
        "uda": axx[1, 2],
    }

    titles = {
        "wind": "$τ$, $∇×τ$",
        "sst": "$T_{10}$",
        "sss": "$S_{10}$",
        "ubot": "$U_{50}$, $ζ_{50}/f$",
        "usurf": "$U_{0}$, $ζ_{0}/f$",
        "uda": "$U_{100}$ \nblue=←,\nred=→",
    }

    vor_kwargs = dict(vmin=-1, vmax=1, add_colorbar=False, cmap=mpl.cm.RdBu_r)

    hct = (sub.curltau / dcpy.oceans.coriolis(sub.curltau.cf["Y"])).plot(
        ax=ax["wind"], add_colorbar=False, cmap=mpl.cm.RdBu_r, vmin=-0.05, vmax=0.05
    )
    inset_colorbar(ax["wind"], hct, **ckwargs)
    sub[["taux", "tauy"]].cf.isel(
        X=slice(None, None, stride), Y=slice(None, None, stride)
    ).plot.quiver(
        x="xh", y="yh", u="taux", v="tauy", scale=2, ax=ax["wind"], add_guide=False
    )

    plot_ts(
        sub.thetao.cf.isel(Z=0).where(sub.D >= 10),
        ax=ax["sst"],
        vmin=23,
        vmax=32,
        levels=np.arange(18, 35, 0.5),
        cmap=mpl.cm.RdYlBu_r,
    )
    plot_ts(
        sub.so.cf.isel(Z=0).where(sub.D >= 10),
        ax=ax["sss"],
        vmin=32,
        vmax=35,
        levels=np.arange(28, 37, 0.25),
        cmap=mpl.cm.Spectral_r,
    )

    hvorsurf = (sub.vor / f0).cf.isel(Z=0).plot(ax=ax["usurf"], **vor_kwargs)
    inset_colorbar(ax["usurf"], hvorsurf, **ckwargs)
    sub[["uo", "vo"]].cf.isel(
        X=slice(None, None, stride),
        Y=slice(None, None, stride),
        Z=-1,
    ).plot.quiver(
        x="xh", y="yh", u="uo", v="vo", scale=15, ax=ax["usurf"], add_guide=False
    )

    hvor50 = (sub.vor / f0).cf.isel(Z=-1).plot(ax=ax["ubot"], **vor_kwargs)
    inset_colorbar(ax["ubot"], hvor50, **ckwargs)
    sub[["uo", "vo"]].cf.isel(
        X=slice(None, None, stride),
        Y=slice(None, None, stride),
        Z=-1,
    ).plot.quiver(
        x="xh", y="yh", u="uo", v="vo", scale=15, ax=ax["ubot"], add_guide=False
    )

    # z_notbl = (sub.D - sub.ePBL_h_ML - sub.bbl_thick_u) / sub.D
    # hbl = z_notbl.plot(
    #    levels=[0, 0.25, 0.5, 0.75, 1],
    #    robust=True,
    #    cmap=mpl.cm.Blues_r,
    #    ax=ax["wind"],
    #    add_colorbar=False,
    #    zorder=-1,
    # )
    # inset_colorbar(ax["wind"], hbl, **ckwargs)

    uda = sub[["uo", "vo"]].mean("zl").where(sub.D <= 80)

    uda.where(uda.uo < 0).cf.isel(
        X=slice(None, None, stride // 2),
        Y=slice(None, None, stride // 2),
    ).plot.quiver(
        x="xh",
        y="yh",
        u="uo",
        v="vo",
        scale=3,
        ax=ax["uda"],
        add_guide=False,
        color="b",
    )
    uda.where(uda.uo > 0).cf.isel(
        X=slice(None, None, stride // 2),
        Y=slice(None, None, stride // 2),
    ).plot.quiver(
        x="xh",
        y="yh",
        u="uo",
        v="vo",
        scale=3,
        ax=ax["uda"],
        add_guide=False,
        color="r",
    )

    [
        ax[name].text(
            x=0.9,
            y=0.9,
            s=title,
            fontsize="medium",
            va="top",
            ha="right",
            transform=ax[name].transAxes,
        )
        for name, title in titles.items()
    ]

    for axxx in axx.flat:
        axxx.plot(
            sub.reeflon, sub.reeflat, color="k", marker="o", ms=4, ls="none", zorder=-1
        )
        sub.D.plot.contour(
            levels=[25, 50, 75, 125, 150, 175],
            colors="k",
            ax=axxx,
            linewidths=0.15,
        )
        sub.D.plot.contour(
            levels=[0, 100, 200, 300],
            colors="k",
            ax=axxx,
            linewidths=0.5,
        )
        axxx.set_xlabel("")
        axxx.set_ylabel("")
        axxx.set_title("")

    dcpy.plots.clean_axes(axx)
    f.set_size_inches((8, 5))
    f.suptitle(f"{sub.time.values}", y=1.025)

    f.savefig(
        f"../movies/tpec_shelf/{sub.time.values}.png", dpi=200, bbox_inches="tight"
    )
    plt.close(f)

    return returnval

In [85]:
def make_plot(fname, func=plot_frame):
    
    import cf_xarray

    rawds = xr.open_dataset(fname, chunks=-1)
    ds = (
        rawds.isel(xh=slice(1, None), yh=slice(1, None))
        .coarsen(time=8)
        .mean(skipna=False)
    )

    geometry = xr.open_dataset(
        f"{dirname}/rundir/ETP.012/run/ocean_geometry.nc"
    ).rename({"lath": "yh", "latq": "yq", "lonh": "xh", "lonq": "xq"})
    geometry = geometry.reindex_like(ds)

    grid = xgcm_grid_mom6(geometry)

    ds["curltau"] = curlz(ds.taux, ds.tauy, grid)
    ds["vor"] = curlz(ds.uo, ds.vo, grid)

    reeflocs = pd.read_csv(
        "/glade/work/kleypas/MOM6/ETP_reef_locs_0.01_lon_lat.csv", names=["lon", "lat"]
    )
    reeflocs["lon"] += 360

    centered = (
        ds.pipe(map_xgcm_func, grid.interp, axis=("X",), to="center").pipe(
            map_xgcm_func, grid.interp, axis=("Y",), to="center"
        )
        # .pipe(map_xgcm_func, grid.interp, axis=("Z",), to="center")
    )
    centered.coords.update(geometry)

    d = (
        centered[["uo", "vo", "thetao", "so", "taux", "tauy", "curltau", "vor", "D"]]
        .cf.sel(Z=[10, 50], method="nearest")
        .cf.sel(X=slice(None, 276), Y=slice(9, None))
    ).compute(scheduler="single-threaded")

    reefsub = reeflocs.query("(lon > 268) & (lon < 276) & (lat > 9) & (lat < 14)")
    d.coords["reeflon"] = ("points", reefsub.lon)
    d.coords["reeflat"] = ("points", reefsub.lat)

    func(d)


%matplotlib inline


make_plot(f"{dirname}/COMPRESSED_OCEAN_SHELF/ocean_shelf_2000_321.nc")

In [86]:
import glob

import dask.delayed

files = sorted(glob.glob(f"{dirname}/COMPRESSED_OCEAN_SHELF/ocean_shelf_*.nc"))

tasks = [dask.delayed(make_plot)(f) for f in files]

In [68]:
cluster.scale(35)

In [87]:
dask.compute(tasks)

([None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,


In [186]:
%matplotlib inline

plot_frame(d.isel(time=100).compute());

KeyboardInterrupt: 

In [185]:
d = (
    centered[
        ["uo", "vo", "thetao", "so", "taux", "tauy", "curltau", "vor", "D"]
    ]
    .chunk({"time": 1})
    .cf.sel(Z=[10, 50], method="nearest")
    .cf.chunk({"Z": -1})
    .cf.sel(X=slice(None, 276), Y=slice(9, None))
    .pipe(dcpy.dask.map_copy)
)
reeflocs = pd.read_csv("/glade/work/kleypas/MOM6/ETP_reef_locs_0.01_lon_lat.csv", names=["lon", "lat"])
reeflocs["lon"] += 360
reefsub = reeflocs.query("(lon > 268) & (lon < 276) & (lat > 9) & (lat < 14)")
d.coords["reeflon"] = ("points", reefsub.lon)
d.coords["reeflat"] = ("points", reefsub.lat)
d = dask.optimize(d)[0]
d = d.chunk({"xh": -1, "yh": -1})
d

<xarray.Dataset>
Dimensions:     (points: 16, time: 6940, xh: 160, yh: 100, zl: 2)
Coordinates: (12/16)
    D           (yh, xh) float64 dask.array<chunksize=(100, 160), meta=np.ndarray>
  * time        (time) datetime64[ns] 1999-12-30T12:00:00 ... 2018-12-29T12:0...
  * zl          (zl) float64 11.25 51.25
  * yh          (yh) float64 9.025 9.075 9.125 9.175 ... 13.82 13.88 13.93 13.98
  * xh          (xh) float64 268.0 268.1 268.1 268.2 ... 275.8 275.9 275.9 276.0
    average_T1  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    dxT         (yh, xh) float64 dask.array<chunksize=(100, 160), meta=np.ndarray>
    dyT         (yh, xh) float64 dask.array<chunksize=(100, 160), meta=np.ndarray>
    Ah          (yh, xh) float64 dask.array<chunksize=(100, 160), meta=np.ndarray>
    wet         (yh, xh) float64 dask.array<chunksize=(100, 160), meta=np.ndarray>
    reeflon     (points) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    reeflat     (points) float64 dask.array<chunksize=(16,), meta=np.ndarray>
Dimensions without coordinates: points
Data variables:
    uo          (time, zl, yh, xh) float32 dask.array<chunksize=(1, 2, 100, 160), meta=np.ndarray>
    vo          (time, zl, yh, xh) float32 dask.array<chunksize=(1, 2, 100, 160), meta=np.ndarray>
    thetao      (time, zl, yh, xh) float32 dask.array<chunksize=(1, 2, 100, 160), meta=np.ndarray>
    so          (time, zl, yh, xh) float32 dask.array<chunksize=(1, 2, 100, 160), meta=np.ndarray>
    taux        (time, yh, xh) float32 dask.array<chunksize=(1, 100, 160), meta=np.ndarray>
    tauy        (time, yh, xh) float32 dask.array<chunksize=(1, 100, 160), meta=np.ndarray>
    curltau     (time, yh, xh) float64 dask.array<chunksize=(1, 100, 160), meta=np.ndarray>
    vor         (time, zl, yh, xh) float64 dask.array<chunksize=(1, 2, 100, 160), meta=np.ndarray>

In [176]:
mapped = d.map_blocks(plot_frame, template=xr.ones_like(d.time).chunk({"time": 1}))

In [177]:
future = mapped.compute()

In [46]:
client.cancel(future)

In [88]:
cluster.close()

In [89]:
client.close()